In [ ]:
# Require auto-mount worker
from google.colab import output
from pathlib import Path
import os, multiprocessing


def megalogin(USERNAME, PASSWORD):
  os.environ['usrC'] = USERNAME
  os.environ['pwdC'] = PASSWORD
  !mega-login $usrC $pwdC


def megainit(USERNAME, PASSWORD):
  if not os.path.exists("/usr/bin/mega-cmd"):
    print("Installing MEGA ...")
    !apt-get -y update')
    !apt-get -y install libmms0 libc-ares2 libc6 libcrypto++6 libgcc1 libmediainfo0v5 libpcre3 libpcrecpp0v5 libssl1.1 libstdc++6 libzen0v5 zlib1g apt-transport-https
    !curl -sL -o /var/cache/apt/archives/MEGAcmd.deb https://mega.nz/linux/MEGAsync/Debian_9.0/amd64/megacmd-Debian_9.0_amd64.deb
    !dpkg -i /var/cache/apt/archives/MEGAcmd.deb
    output.clear()
    print("MEGA is installed.")
  print('----- Login in -----')
  p = multiprocessing.Process(target=megalogin, args=[USERNAME, PASSWORD, ])
  p.start()
  p.join(10)
  if p.is_alive():
    p.terminate()
    p.join()
  print('----- Completed -----')
  !mega-whoami


def megatransfer(FROM, TO, PARAS):
  print('----- Checking -----')
  char = list(PARAS)
  alpha = (char[0] != '%')
  !mega-find '' | awk -F"/" '{OFS="/"; NF=(NF-1);print $0}' | sort | uniq > /content/folder.txt
  with open('/content/folder.txt', 'r') as f:
    folders = []
    for line in f:
      if line != '\n':
        folders.append(line.strip('\n'))
  !mega-find '{FROM}' > /content/file.txt
  with open('/content/file.txt', 'r') as f:
    files = []
    for line in f:
      raw = line.strip('\n')
      if not raw in folders:
        f_folder, f_file = os.path.split(raw)
        if alpha:
          if f_file[:1].lower() in char:
            files.append(raw)
        else:
          if not f_file[:1].isalpha():
            files.append(raw)
  print('----- Trasfering -----')
  numfile = len(files)
  num = 0
  for src in files:
    folder_path, file_path = os.path.split(src)
    dst = os.path.join(TO, folder_path.lstrip(FROM).lstrip('/'))
    if not os.path.isdir(dst):
      Path(dst).mkdir(parents=True, exist_ok=True)
    num += 1
    print('Transfering:', num, '/', numfile)
    if not os.path.exists(os.path.join(dst, file_path)):
      !mega-get '{src}' '{dst}'


def mega(PARA):
  if len(PARA) < 5:
    print('PARA corrupted')
    return
  USERNAME = PARA[0]
  PASSWORD = PARA[1]
  FROM = PARA[2]
  TO = PARA[3]
  PARAS = PARA[4]
  megainit(USERNAME, PASSWORD)
  megatransfer(FROM, TO, PARAS)
